# Self Organized Maps (SOMs)

## Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import numpy as np
import pandas as pd

# SimpSOM, https://simpsom.readthedocs.io
import simpsom as sps
# from simpsom.plots import scatter_on_map

# Astropy
from astropy.table import Table

# CuPY
import cupy as cp

## Import catalog

In [ ]:
# RADIUS (3''), EMU (1''), DESY6GOLD (0.1''), VIKINGDR5 (0.1''), CATWISE2020 (0.2''), MAGNITUDES
# For a false detection rate of <3% 
# p_i>=0.1 & match_flag==1 & p_any>0.85
emu_matched_catalog_path_1 = '/data/mfonseca/cross_match/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_Mags/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_Mags.fits'

In [2]:
emu_matched_catalog_path = '/data/mfonseca/cross_match/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_noMags/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_fakematch.fits'
emu_matched_catalog = Table.read(emu_matched_catalog_path)
emu_matched_catalog = emu_matched_catalog.to_pandas()

In [ ]:
emu_matched_catalog.describe()

In [ ]:
# Here we filter the NWAY catalog to keep only the best objects

# From NWAY
# Filter the catalog based on the match_flag and p_i, p_any values
match_mask = (emu_matched_catalog['match_flag'] == 1) & (emu_matched_catalog['p_i'] >= 0.1) & (emu_matched_catalog['p_any'] >=0.81)
catalog_matched = emu_matched_catalog[match_mask]
print(f'Number of objects in the catalog {len(catalog_matched)}')

In [ ]:
# Here we select objects that have matches in all three surveys
# full_match_mask = (catalog_matched['DES_coadd_object_id'] != -99) & (catalog_matched['VKG_sourceID'] != -99) & (catalog_matched['CAT_source_id'] != -99)
full_match_mask = (catalog_matched['DESY6_coadd_object_id'] != -99) & (catalog_matched['VKG_sourceID'] != -99) & (catalog_matched['CAT_source_id'] != -99)
catalog_matched = catalog_matched[full_match_mask]
print(f'Number of objects in the catalog after filtering {len(catalog_matched)}')

In [ ]:
# Filter the multiwavelenght catalog to keep only the columns needed
emu_columns = ['EMU_island_id']

# desdr2_mag = ['DES_mag_auto_g_dered', 'DES_mag_auto_r_dered', 'DES_mag_auto_i_dered', 'DES_mag_auto_z_dered']
# desdr2_colors = ['DES_g_r_dered', 'DES_r_i_dered', 'DES_i_z_dered']

desy6gold_mag = ['DESY6_mag_auto_g_extcorr', 'DESY6_mag_auto_r_extcorr', 'DESY6_mag_auto_i_extcorr', 'DESY6_mag_auto_z_extcorr', 'DESY6_mag_auto_y_extcorr']
desy6gold_colors = ['DESY6_g_r_extcorr', 'DESY6_r_i_extcorr', 'DESY6_i_z_extcorr', 'DESY6_z_y_extcorr']

viking_mag = ['VKG_zAperMag3_ab_extcorr', 'VKG_jAperMag3_ab_extcorr', 'VKG_yAperMag3_ab_extcorr', 'VKG_ksAperMag3_ab_extcorr', 'VKG_hAperMag3_ab_extcorr']
viking_colors = ['VKG_z_y_am3_extcorr', 'VKG_y_j_am3_extcorr', 'VKG_j_h_am3_extcorr', 'VKG_h_ks_am3_extcorr']

catwise_mag = ['CAT_W1mpro_ab', 'CAT_W2mpro_ab']
catwise_colors = ['CAT_w1_w2_ab']

magnitude_columns = emu_columns + desy6gold_colors + viking_colors

catalog_matched = catalog_matched[magnitude_columns]

# Should be the same lenght as the filtered catalog
print(len(catalog_matched))

In [ ]:
# Histograms of each columns

# for col in catalog_filtered_magnitude.columns:
#     plt.figure(figsize=(10, 5))
#     plt.hist(catalog_filtered_magnitude[col], bins=100, color='blue', alpha=0.7)
#     plt.title(f'Histogram of {col}')
#     plt.xlabel(col)
#     plt.ylabel('Frequency')
#     plt.grid()
#     plt.show()

## Define the Topology

Define the topology of the SOM, follow indications from "Self-Organizing Maps and Their Applications to Data Analysis" R. Ponmalai, C. Kamath

In [ ]:
# SOM topology (lattice size)

# Number of sources in our dataframe
inputdata_size = len(catalog_matched)

# From Kohonen, the optimal ratio of height to width of the lattice
# is equal to the ratio of the two largest eigenvalue of the autocorrelation matrix
cov_std = catalog_matched.cov()
eigen_values = np.linalg.eigvals(cov_std.values)
eigen_values_sorted = np.sort(eigen_values)[::-1]
ratio_eigenvalues = eigen_values_sorted[0] / eigen_values_sorted[1]

# From Kohonen, the number of nodes if 5*sqrt(n), where n is the number of data points
som_dim = int(np.sqrt(inputdata_size) * 5)

# The number of nodes in the x and y direction
som_x = int(np.sqrt(som_dim * ratio_eigenvalues))
som_y = int(som_x / ratio_eigenvalues)

#=================================================================

# Print the results
print('Dimension of the SOM: ', som_dim)
print('Square SOM map size ', np.round(np.sqrt(som_dim)))
print('SOM map size using eigenvalues: ', som_x, 'x', som_y)

In [ ]:
# Resize the SOM, if wanted, keeping the eigenvalue ratio

# som_x = 
# som_y = int(som_x / ratio_eigenvalues)
# som_dim = som_x * som_y

#=================================================================

# Print the results
print('Dimension of the SOM: ', som_dim)
print('Square SOM map size ', np.round(np.sqrt(som_dim)))
print('SOM map size using eigenvalues: ', som_x, 'x', som_y)

In [ ]:
# Definine the subset of the catalog to use
data_fraction2use = 1
data = catalog_matched.sample(frac=data_fraction2use)

# Normalize the data
data.to_numpy()
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
data = data.values 

## Quantization Error (QE) and Topographic Error (TE)

In [ ]:
# THIS FUNCTION WORKS

# Quantization Error (QE)
# Average distance of a data point to the nearest lattice node
# Measures how well the mapping fits the distribution of the data

def som_quantization_error(data, trained_som):
    '''
    Calculates the quantization error of a trained SOM.ipynb

    Args:
        data: NumPy array of input data.
        trainded_som: trained SimpSOM SOMNet object

    Returns:
        quantization error (float)
    '''
    # Convert to clean NumPy array with uniform dtype first
    data_np = np.asarray(data, dtype=np.float64)

    # Convert NumPy array to CuPy array for GPU processing
    data_cp = cp.array(data_np)

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Get the weights of the BMUs
    bmu_weights = cp.array([trained_som.nodes_list[int(bmu_idx)].weights for bmu_idx in bmu_indices])

    # Vectorized distance calculation
    distances = cp.linalg.norm(bmu_weights - data_cp, axis=1)
    total_distance = cp.sum(distances)

    quantization_error = float(total_distance / len(data))
    return quantization_error


In [ ]:
# THIS FUNCTION WORKS

# Topographic Error (TE)
# Proportion of data points whose BMU and second BMU are NOT neighbors
# Measures how well the shape of the data is preserved in the output space

def som_topographic_error(data, trained_som):
    """
    Computes topographic error ET for a SOM with hex topology using GPU (CuPy).
    
    Args:
        data:    The input data used to train the SOM (NumPy array).
        trained_som: A trained SOMNet object from the simpsom library.

    Returns:
        Topographic error (float between 0 and 1).
        Proportion of data points whose BMU and second BMU are NOT neighbors
    """
    # Convert to clean NumPy array with uniform dtype first
    data_np = np.asarray(data, dtype=np.float64)

    # Convert data to CuPy arrays
    data_cp = cp.array(data)

    # Get all the nodes' weights from the trained SOM (GPU-enabled)
    weights_cp = cp.array([node.weights for node in trained_som.nodes_list])

    # Initialize total error
    total_error = 0

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Find all the second BMU indices
    som_dist = sps.distances.Distance(xp=np) # Initialize the distance object, as required by the documentation
    distances = som_dist.pairdist(data_cp, weights_cp, metric='euclidean')

    sorted_indices = cp.argsort(distances, axis=1)
    sbmu_indices = sorted_indices[:, 1]


    # Get the positions of the BMU and second BMU in the grid
    bmu_positions = cp.array([trained_som.nodes_list[int(bmu_idx)].pos for bmu_idx in bmu_indices])
    sbmu_positions = cp.array([trained_som.nodes_list[int(sbmu_idx)].pos for sbmu_idx in sbmu_indices])


    bmu_row, bmu_col = bmu_positions[:, 0], bmu_positions[:, 1]
    sbmu_row, sbmu_col = sbmu_positions[:, 0], sbmu_positions[:, 1]


    # Check if BMU and sBMU are neighbors
    row_neighbors = cp.abs(bmu_row - sbmu_row)
    col_neighbors = cp.abs(bmu_col - sbmu_col)

    not_neighbors = (row_neighbors > 1) | (col_neighbors > 1)

    number_not_neighbors = cp.sum(not_neighbors)

    # Compute the topographic error
    topographic_error = number_not_neighbors / data_cp.shape[0]

    return float(topographic_error)

# Running the SOM

In [ ]:
# Does not work correcly because the learning rate is not updated in each training epoch

def track_som_errors(data, epochs, net_height=20, net_width=20):
    som_net = sps.SOMNet(
        net_height=net_height,
        net_width=net_width,
        data=data,
        topology='hexagonal',
        init='PCA',
        metric='euclidean',
        neighborhood_fun='gaussian',
        PBC=True,
        GPU=True,
        random_seed=42,
        output_path="./"
    )

    quantization_errors = []
    topographic_errors = []

    for epoch in range(epochs):

        som_net.train(
            train_algo='batch',
            epochs=1,
            start_learning_rate = 0.1,
            early_stop=None,
            batch_size=-1
        )

        quant_error = som_quantization_error(data, som_net)
        topo_error = som_topographic_error(data, som_net)

        quantization_errors.append(quant_error)
        topographic_errors.append(topo_error)

        print(f"Epoch {epoch+1}/{epochs} | QE: {quant_error:.4f} | TE: {topo_error:.4f}")

    return quantization_errors, topographic_errors, som_net

In [ ]:
# Define the SOM
som = sps.SOMNet(
    som_x,
    som_y,
    data,
    load_file = None,
    metric= 'euclidean', # 'euclidian' or 'cosine' or 'manhattan'
    # metrick_kwds={},
    init = 'random' , # 'random' or 'pca'
    PBC = True,
    GPU = True,
    random_seed = 69,
    topology='hexagonal', # 'square' or 'hexagonal'
    debug= True
)

som.train(
    train_algo = 'batch', 
    epochs = 200,
    # start_learning_rate = 0.01 , # Used only for online training
    early_stop = None ,
    early_stop_patience = 5 ,
    early_stop_tolerance = 0.01 ,
    batch_size = -1
)

In [ ]:
som.save_map('/data/mfonseca/soms/test1.npy')

## U-Matrix

In [ ]:
som.plot_map_by_difference(show=True, print_out=True)

## Plot Quantization Error and 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(net.quantization_error, label='Quantization Error', color='blue')
plt.plot(net.topographic_error, label='Topographic Error', color='orange')
plt.title('SOM Training Errors')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.grid()
plt.show()

## Component Planes

In [ ]:
for i in range(len(catalog_filtered_magnitude.columns)):
    som.plot_map_by_feature(
        feature_ix= i,
        show=True,
        print_out=True,
        cbar_label=catalog_filtered_magnitude.columns[i],
    )

# MiniSom

### Example

In [ ]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt', 
                    names=['area', 'perimeter', 'compactness', 'length_kernel', 'width_kernel',
                   'asymmetry_coefficient', 'length_kernel_groove', 'target'], usecols=[0, 5], 
                   sep='\t+', engine='python')
# data normalization
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
data = data.values


# Initialization and training
som_shape = (1, 3)
som = MiniSom(som_shape[0], som_shape[1], data.shape[1], sigma=.5, learning_rate=.5,
              neighborhood_function='gaussian', random_seed=10)

som.train_batch(data, 500, verbose=True)

In [ ]:
# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in data]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)

In [ ]:
# plotting the clusters using the first 2 dimentions of the data
for c in np.unique(cluster_index):
    plt.scatter(data[cluster_index == c, 0],
                data[cluster_index == c, 1], label='cluster='+str(c), alpha=.7)

# plotting centroids
for centroid in som.get_weights():
    plt.scatter(centroid[:, 0], centroid[:, 1], marker='x', 
                s=4, linewidths=10, color='k', label='centroid')
plt.legend(loc='best')

### With our galaxy values

In [ ]:
galaxy_names = catalog_df['EMU_island_name'].str.decode('utf-8').str.strip()

# Normalize the data
data = (catalog_df_filtered_matches_magnitude - np.mean(catalog_df_filtered_matches_magnitude, axis=0)) / np.std(catalog_df_filtered_matches_magnitude, axis=0)
data = data.values

In [ ]:
som_shape = (10, 10)
som = MiniSom(som_shape[0], som_shape[1], data.shape[1], sigma=.5, learning_rate=.1,
              neighborhood_function='gaussian', random_seed=0)

som.train_batch(data, 100000, verbose=True)

In [ ]:
plt.figure(figsize=(7, 7))
frequencies = som.activation_response(data)
plt.pcolor(frequencies.T, cmap='Blues') 
plt.colorbar()
plt.show()

In [ ]:
# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in data]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)

In [ ]:
# plotting the clusters using the first 2 dimentions of the data
for c in np.unique(cluster_index):
    plt.scatter(data[cluster_index == c, 0],
                data[cluster_index == c, 1], label='cluster='+str(c), alpha=.7)

# plt.xlim(-.5,1)
# plt.ylim(-1,1)
